In [ ]:
# Mount Google Drive and define time function #
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
COLAB = True
print("Note: using Google CoLab")

import time 

def time_elapsed(sec):
  h = int(sec/3600)
  m = int(sec/60)
  s = sec % 60
  return "{}:{>02}:{:>05.2f}".format(h,m,s)

# Read Titanic Data #
import pandas as pd
from scipy.stats import zscore
import numpy as np
from tqdm import tqdm
import os 

DATA_PATH = '/content/drive/My Drive/projects/titanic'
TRAIN_FILE_PATH = os.path.join(DATA_PATH, 'train.csv') 
TEST_FILE_PATH = os.path.join(DATA_PATH, 'test.csv') 

df = pd.read_csv(TRAIN_FILE_PATH,na_values=['NA','?'])
df_test = pd.read_csv(TEST_FILE_PATH,na_values=['NA','?'])

# Create a Title Column #
df.insert(2, "Title",'') 

df.loc[df.Name.str.contains(pat = 'Mr'),'Title']='Mr' 
df.loc[df.Name.str.contains(pat = 'Master'),'Title']='Master' 
df.loc[df.Name.str.contains(pat = 'Miss'),'Title']='Miss' 
df.loc[df.Name.str.contains(pat = 'Mrs'),'Title']='Mrs' 

# filling age gaps #
# Age depends on title and parch #

# get median age of female childreen # Title Miss means unmarried woman and Parch > 0 means traveling with parents/childreen
female_Miss_childreen = df[(df.Title=='Miss') & (df.Parch > 0) ]["Age"].median()
female_Miss_adult = df[(df.Title=='Miss') & (df.Parch == 0) ]["Age"].median() # female adult without husband
female_Mrs_adult = df[(df.Title=='Mrs')]["Age"].median() # female adult with husband
male_Master = df[(df.Title=='Master')]["Age"].median() # male child
male_Mr = df[(df.Title=='Mr')]["Age"].median()                 
# fill those empty age gaps
df.loc[(df.Title=='Miss') & (df.Parch > 0) & (df.Age.isnull()), "Age"] = female_Miss_childreen
df.loc[(df.Title=='Miss') & (df.Parch == 0) & (df.Age.isnull()), "Age"] = female_Miss_adult
df.loc[(df.Title=='Mrs') & (df.Age.isnull()), "Age"] = female_Mrs_adult
df.loc[(df.Title=='Mr') & (df.Age.isnull()), "Age"] = male_Mr
df.loc[(df.Title=='Master') & (df.Age.isnull()), "Age"] = male_Master
df.loc[(df.Age.isnull()), "Age"] = male_Master
# Age is not much important factor while you are above 18, so lets make column to separate kids from adults #
# Column named "Adult" 1 means age is above 18, 0-belov
df.insert(2, "Adult",'') 
df.loc[(df.Age > 18), "Adult"] = 1
df.loc[(df.Age <= 18), "Adult"] = 0
df.loc[(df.Age.isnull()), "Adult"] = 1
df["Adult"] = pd.to_numeric(df.Adult, errors='coerce')

# Important factor is to survive is Fare, more expensive ticket more chance to survive
# but some people have the same ticket so need to divide their Fare to numbers of people sharing their ticket
# then need to fill missing Fare gaps acording to Pclass they belong

# Adding column Fare per Person #
df.insert(12, "FarePP",0) 

# Evaluating Fare PP #
tickets_count = df.groupby("Ticket")["Ticket"].count() #counting tickets
for ticket_number in tickets_count.keys():
  fpp = (df[df.Ticket == ticket_number]["Fare"]) / tickets_count[ticket_number]
  df.loc[df.Ticket == ticket_number, "FarePP"] = fpp

# filling gaps
FarePP_1_class = df[(df.Pclass==1) ]["FarePP"].median()
FarePP_2_class = df[(df.Pclass==2) ]["FarePP"].median()
FarePP_3_class = df[(df.Pclass==3) ]["FarePP"].median()

df.loc[(df.Pclass==1) & (df.FarePP.isnull()), "FarePP"] = FarePP_1_class
df.loc[(df.Pclass==2) & (df.FarePP.isnull()), "FarePP"] = FarePP_2_class
df.loc[(df.Pclass==3) & (df.FarePP.isnull()), "FarePP"] = FarePP_3_class

df = pd.concat([df,pd.get_dummies(df['Sex'],prefix="Sex")],axis=1)
df.drop('Sex', axis=1, inplace=True)
df = pd.concat([df,pd.get_dummies(df['Embarked'],prefix="Embarked")],axis=1)
df.drop('Embarked', axis=1, inplace=True)
df['FarePP'] = zscore(df['FarePP'])
df['Age'] = zscore(df['Age'])
# decide which columns are to drop
y = df['Survived'].values
x_columns = df.columns.drop('PassengerId').drop('Survived').drop('Title').drop('Ticket').drop('Fare').drop('Cabin').drop('Name').drop('Age')
x = df[x_columns].values

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
import pandas as pd
import os
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.optimizers import Adam

def generate_model(dropout, neuronPct, neuronShrink):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 1000)
    
    # Construct neural network
    # kernel_initializer = tensorflow.keras.initializers.he_uniform(seed=None)
    model = Sequential()

    # So long as there would have been at least 25 neurons and fewer than 10
    # layers, create a new layer.
    layer = 0
    while neuronCount>25 and layer<4:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
            model.add(Dense(neuronCount, 
                input_dim=x.shape[1], 
                activation=PReLU()))
        else:
            model.add(Dense(neuronCount, activation=PReLU())) 
        layer += 1

        # Add dropout after each hidden layer
        model.add(Dropout(dropout))

        # Shrink neuron count for each layer
        neuronCount = neuronCount * neuronShrink

    model.add(Dense(1,activation='sigmoid')) # Output
    return model

In [ ]:
  
def evaluate_network(dropout,lr,neuronPct,neuronShrink):
    SPLITS = 2

    # Bootstrap
    boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)

    # Track progress
    mean_benchmark = []
    epochs_needed = []
    num = 0
    

    # Loop through samples
    for train, test in boot.split(x,y):
        start_time = time.time()
        num+=1

        # Split train and test
        x_train = x[train]
        y_train = y[train]
        x_test = x[test]
        y_test = y[test]
        y_test
        model = generate_model(dropout, neuronPct, neuronShrink)
        model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=50, verbose=0, mode='auto', restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train,y_train,validation_data=(x_test,y_test),
                  callbacks=[monitor],verbose=0,epochs=200)
        epochs = monitor.stopped_epoch
        epochs_needed.append(epochs)

        # Predict on the out of boot (validation)
        pred = model.predict(x_test)

         #Measure this bootstrap's log loss
        y_compare = y_test # For log loss calculation

        y_compare = np.array(y_test)
        y_compare = y_compare.astype('float32')
        y_compare = np.round(y_compare)
        score_temp = np.zeros(len(y_test))
        for i in range(len(y_test)):
          score_temp[i] = math.sqrt((y_compare[i]-pred[i])*(y_compare[i]-pred[i]))
        score = sum(score_temp)
        mean_benchmark.append(score)
        m1 = statistics.mean(mean_benchmark)
       
        m2 = statistics.mean(epochs_needed)
        mdev = statistics.pstdev(mean_benchmark)

        # Record this iteration
        time_took = time.time() - start_time
        
    tensorflow.keras.backend.clear_session()
    return (-m1)



In [ ]:
from bayes_opt import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

# Bounded region of parameter space
pbounds = {'dropout': (0.0, 0.499),
           'lr': (0.000001, 0.1),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.1, 1)
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

start_time = time.time()
optimizer.maximize(init_points=4, n_iter=20,)
time_took = time.time() - start_time

print(f"Total runtime: {hms_string(time_took)}")
print(optimizer.max)

|   iter    |  target   |  dropout  |    lr     | neuronPct | neuron... |
-------------------------------------------------------------------------
|  1        | -25.14    |  0.2081   |  0.07203  |  0.01011  |  0.3721   |
|  2        | -22.5     |  0.07323  |  0.009235 |  0.1944   |  0.411    |
|  3        | -20.04    |  0.198    |  0.05388  |  0.425    |  0.7167   |
|  4        | -27.43    |  0.102    |  0.08781  |  0.03711  |  0.7034   |
|  5        | -23.44    |  0.003852 |  0.01619  |  0.7123   |  0.7883   |
|  6        | -24.43    |  0.2208   |  0.04135  |  0.5523   |  0.7698   |
|  7        | -29.58    |  0.3111   |  0.02232  |  0.9595   |  0.8242   |
|  8        | -17.72    |  0.01058  |  0.08079  |  0.9652   |  0.7319   |
|  9        | -28.67    |  0.1818   |  0.002241 |  0.1609   |  0.9744   |
|  10       | -24.44    |  0.4967   |  0.03252  |  0.8187   |  0.1402   |
|  11       | -30.08    |  0.4896   |  0.02704  |  0.8107   |  0.5019   |
|  12       | -25.51    |  0.1621   | 

NameError: ignored